In [ ]:
from OpenGL.GLUT import *
from OpenGL.GLU import *
from OpenGL.GL import *
import sys

name = 'ball_glut'

def main():
    glutInit(sys.argv)
    glutInitDisplayMode(GLUT_DOUBLE | GLUT_RGB | GLUT_DEPTH)
    glutInitWindowSize(400,400)
    glutCreateWindow(name)

    glClearColor(0.,0.,0.,1.)
    glShadeModel(GL_SMOOTH)
    glEnable(GL_CULL_FACE)
    glEnable(GL_DEPTH_TEST)
    glEnable(GL_LIGHTING)
    lightZeroPosition = [10.,4.,10.,1.]
    lightZeroColor = [0.8,1.0,0.8,1.0] #green tinged
    glLightfv(GL_LIGHT0, GL_POSITION, lightZeroPosition)
    glLightfv(GL_LIGHT0, GL_DIFFUSE, lightZeroColor)
    glLightf(GL_LIGHT0, GL_CONSTANT_ATTENUATION, 0.1)
    glLightf(GL_LIGHT0, GL_LINEAR_ATTENUATION, 0.05)
    glEnable(GL_LIGHT0)
    glutDisplayFunc(display)
    glMatrixMode(GL_PROJECTION)
    gluPerspective(40.,1.,1.,40.)
    glMatrixMode(GL_MODELVIEW)
    gluLookAt(0,0,10,
              0,0,0,
              0,1,0)
    glPushMatrix()
    glutMainLoop()
    return

def display():
    glClear(GL_COLOR_BUFFER_BIT|GL_DEPTH_BUFFER_BIT)
    glPushMatrix()
    color = [1.0,0.,0.,1.]
    glMaterialfv(GL_FRONT,GL_DIFFUSE,color)
    glutSolidSphere(2,20,20)
    glPopMatrix()
    glutSwapBuffers()
    return

if __name__ == '__main__': main()

In [ ]:
from OpenGL.GL import *
from OpenGL.GLUT import *
from OpenGL.GLU import *

class Point:
    def __init__(self,x,y):
        self.x=x
        self.y=y

points=[]

def initFun():
    glClearColor(1.0,1.0,1.0,0.0)
    glColor3f(0.0,0.0, 0.0)
    glMatrixMode(GL_PROJECTION)
    glLoadIdentity()
    gluOrtho2D(0.0,640.0,0.0,480.0)
    

def displayFun():
    global points
    glClear(GL_COLOR_BUFFER_BIT)
    glBegin(GL_LINE_STRIP)
    glColor3f(0,0,0)
    for p in points:
        glVertex2i(p.x,p.y)
    glEnd()
    glFlush()

def mouseFun(button,state,x,y):
    global points
    if button==GLUT_LEFT_BUTTON and state == GLUT_DOWN:
        p = Point(x,480-y)
        points.append(p)
    if button==GLUT_RIGHT_BUTTON and state == GLUT_DOWN:
        if len(points) != 0:
            points=points[:-1]
    glutPostRedisplay()
        
if __name__ == '__main__':
    glutInit()
    glutInitWindowSize(640,480)
    glutCreateWindow("Polyline")
    glutInitDisplayMode(GLUT_SINGLE | GLUT_RGB)
    glutDisplayFunc(displayFun)
    glutMouseFunc(mouseFun)
    initFun()
    glutMainLoop()

In [ ]:
#!/usr/bin/python
from OpenGL.GLUT import *
from OpenGL.GLU import *
from OpenGL.GL import *

name = "Hello, World"
height = 400
width = 400
rotate = 0
beginx = 0.
beginy = 0.
rotx = 0.
roty = 0.

def display():
    glClear(GL_COLOR_BUFFER_BIT|GL_DEPTH_BUFFER_BIT)
    glLoadIdentity()
    gluLookAt(0,0,10,0,0,0,0,1,0)
    glRotatef(roty,0,1,0)
    glRotatef(rotx,1,0,0)
    glCallList(1)
    glutSwapBuffers()
    return

def mouse(button,state,x,y):
    global beginx,beginy,rotate
    if button == GLUT_LEFT_BUTTON and state == GLUT_DOWN:
        rotate = 1
        beginx = x
        beginy = y
    if button == GLUT_LEFT_BUTTON and state == GLUT_UP:
        rotate = 0
    return

def motion(x,y):
    global rotx,roty,beginx,beginy,rotate
    if rotate:
        rotx = rotx + (y - beginy)
        roty = roty + (x - beginx)
        beginx = x
        beginy = y
        glutPostRedisplay()
    return

def keyboard():
    return

glutInit(name)
glutInitDisplayMode(GLUT_DOUBLE | GLUT_RGB | GLUT_DEPTH)
glutInitWindowSize(height,width)
glutCreateWindow(name)
glClearColor(0.,0.,0.,1.)

# setup display list
glNewList(1,GL_COMPILE)
glPushMatrix()
glTranslatef(0.,1.,0.) #move to where we want to put object
glutSolidSphere(1.,20,20) # make radius 1 sphere of res 10x10
glPopMatrix()
glPushMatrix()
glTranslatef(0.,-1.,0.) #move to where we want to put object
glutSolidSphere(1.,20,20) # make radius 1 sphere of res 10x10
glPopMatrix()
glEndList()

#setup lighting
glEnable(GL_CULL_FACE)
glEnable(GL_DEPTH_TEST)
glEnable(GL_LIGHTING)
lightZeroPosition = [10.,4.,10.,1.]
lightZeroColor = [0.8,1.0,0.8,1.0] # greenish
glLightfv(GL_LIGHT0, GL_POSITION, lightZeroPosition)
glLightfv(GL_LIGHT0, GL_DIFFUSE, lightZeroColor)
glLightf(GL_LIGHT0, GL_CONSTANT_ATTENUATION, 0.1)
glLightf(GL_LIGHT0, GL_LINEAR_ATTENUATION, 0.05)
glEnable(GL_LIGHT0)

#setup cameras
glMatrixMode(GL_PROJECTION)
gluPerspective(40.,1.,1.,40.)
glMatrixMode(GL_MODELVIEW)
gluLookAt(0,0,10,0,0,0,0,1,0)
glPushMatrix()

#setup callbacks
glutDisplayFunc(display)
glutMouseFunc(mouse)
glutMotionFunc(motion)
glutKeyboardFunc(keyboard)


glutMainLoop()

In [ ]:
#!/usr/bin/python
from OpenGL.GLUT import *
from OpenGL.GLU import *
from OpenGL.GL import *

def I2D(ni,i,j):
    return ni * j + i

ni=512
nj=256
    
def display():
    # set upper and lower limits for plotting
    minvar=0.0
    maxvar=0.5

    # do one Lattice Boltzmann step: stream, BC, collide:
    #stream()
    #apply_BCs()
    #collide()

    # convert the plotvar array into an array of colors to plot
    # if the mesh point is solid, make it black
    for j in xrange(nj):
        for i in xrange(ni):
            i0 = I2D(ni,i,j)
            frac = (plotvar[i0]-minvar)/(maxvar-minvar)
            icol = frac * ncol
            isol = solid[i0]
            plot_rgba[i0] = isol * cmap_rgba[icol] 


    # Fill the pixel buffer with the plot_rgba array
    glBufferData(
        GL_PIXEL_UNPACK_BUFFER_ARB,
        ni * nj, 
        plot_rgba,
        GL_STREAM_COPY
    )

    # Copy the pixel buffer to the texture, ready to display
    glTexSubImage2D(GL_TEXTURE_2D,0,0,0,ni,nj,GL_RGBA,GL_UNSIGNED_BYTE,0)

    # Render one quad to the screen and colour it using our texture
    # i.e. plot our plotvar data to the screen
    glClear(GL_COLOR_BUFFER_BIT)
    glBegin(GL_QUADS)
    glTexCoord2f (0.0, 0.0)
    glVertex3f (0.0, 0.0, 0.0)
    glTexCoord2f (1.0, 0.0)
    glVertex3f (ni, 0.0, 0.0)
    glTexCoord2f (1.0, 1.0)
    glVertex3f (ni, nj, 0.0)
    glTexCoord2f (0.0, 1.0)
    glVertex3f (0.0, nj, 0.0)
    glEnd()
    glutSwapBuffers()


def resize(w,h):
    # GLUT resize callback to allow us to change the window size
    width = w
    height = h
    glViewport (0, 0, w, h)
    glMatrixMode (GL_PROJECTION)
    glLoadIdentity()
    glOrtho(0., ni, 0., nj, -200. ,200.)
    glMatrixMode(GL_MODELVIEW)
    glLoadIdentity()

    
def mouse(button,state,x,y):
    global draw_solid_flag, ipos_old, jpos_old
    if button == GLUT_LEFT_BUTTON and state == GLUT_DOWN:       
        draw_solid_flag = 0
        xx = x
        yy = y
        ipos_old = xx / width * ni
        jpos_old = (height-yy) / height * nj
        
    if button == GLUT_LEFT_BUTTON and state == GLUT_UP:        
        draw_solid_flag = 1
        xx = x
        yy = y
        ipos_old = xx / width * ni
        jpos_old = (height-yy) / height * nj

        
def mouse_motion(x,y):
    '''
    GLUT call back for when the mouse is moving
    This sets the solid array to draw_solid_flag as set in the mouse callback
    It will draw a staircase line if we move more than one pixel since the
    last callback - that makes the coding a bit cumbersome:
    '''
    global ipos_old, jpos_old
    
    xx = x
    yy = y
    ipos = xx / width * ni
    jpos = (height-yy) / height * nj

    if ipos <= ipos_old:
        i1 = ipos
        i2 = ipos_old
        j1 = jpos
        j2 = jpos_old
    
    else:
        i1 = ipos_old
        i2 = ipos
        j1 = jpos_old
        j2 = jpos
    
    jlast = j1

    for i in xrange(i1, i2+1):
        if i1 != i2:
            frac = (i-i1) / (i2-i1)
            jnext = (frac * (j2-j1)) + j1        
        else:
            jnext=j2
        
        if jnext >= jlast:
            solid[I2D(ni,i,jlast)] = draw_solid_flag
            
            for j in xrange(jlast, jnext+1):
                solid[I2D(ni,i,j)] = draw_solid_flag       
        
        else:
            solid[I2D(ni,i,jlast)] = draw_solid_flag
            
            for j in xrange(jnext, jlast+1):
                solid[I2D(ni,i,j)] = draw_solid_flag            
        
        jlast = jnext
    
    ipos_old=ipos
    jpos_old=jpos



glutInit('2D LBM')
glutInitDisplayMode(GLUT_DOUBLE | GLUT_RGB)
glutInitWindowSize(ni, nj)
glutInitWindowPosition(50, 50)
glutCreateWindow('2D LBM')

glClearColor(0.0,0.0,0.0,0.0)
glMatrixMode(GL_PROJECTION)
glLoadIdentity()
glOrtho(0,ni,0.,nj, -200.0, 200.0)

glEnable(GL_TEXTURE_2D)

gl_Tex = glGenTextures(1)
glBindTexture(GL_TEXTURE_2D, gl_Tex)

glTexParameteri(GL_TEXTURE_2D, GL_TEXTURE_WRAP_S, GL_CLAMP)
glTexParameteri(GL_TEXTURE_2D, GL_TEXTURE_WRAP_T, GL_CLAMP)
glTexParameteri(GL_TEXTURE_2D, GL_TEXTURE_MAG_FILTER, GL_LINEAR)
glTexParameteri(GL_TEXTURE_2D, GL_TEXTURE_MIN_FILTER, GL_LINEAR)
glTexImage2D(GL_TEXTURE_2D, 0, GL_RGBA8, ni, nj, 0, GL_RGBA, GL_UNSIGNED_BYTE, None)

#gl_PBO = glGenBuffers(1)
#glBindBuffer(GL_PIXEL_UNPACK_BUFFER_ARB, gl_PBO)

#setup callbacks
glutDisplayFunc(display)
glutReshapeFunc(resize)
#glutIdleFunc(display)
#glutMouseFunc(mouse)
#glutMotionFunc(mouse_motion)

glutMainLoop()